<center><h1>Kaggle Challenge</h1>

<h3>1. Reading the Training and Testing Data</h3>
                                           

In [23]:
import pandas as pd
import requests


df = pd.read_csv(r'C:\Users\udit sharma\Desktop\MSA\sentiment-analysis-msa-phase-2\train.csv')
df_test = pd.read_csv(r'C:\Users\udit sharma\Desktop\MSA\sentiment-analysis-msa-phase-2\test.csv')
df.head()

,textID,text,sentiment
0,cb774db0d1,"I`d have responded, if I were going",neutral
1,549e992a42,Sooo SAD I will miss you here in San Diego!!!,negative
2,088c60f138,my boss is bullying me...,negative
3,9642c003ef,what interview! leave me alone,negative
4,358bd9e861,"Sons of ****, why couldn`t they put them on t...",negative


In [24]:
import nltk
from nltk.corpus import movie_reviews, stopwords
from nltk.stem import WordNetLemmatizer
from nltk.tokenize import RegexpTokenizer
from sklearn.model_selection import train_test_split, cross_val_score, cross_val_predict, GridSearchCV
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import SGDClassifier
from sklearn.pipeline import Pipeline
from sklearn.metrics import confusion_matrix, accuracy_score

Total Count of value

In [25]:
df['sentiment'].value_counts()
#27481

neutral     11118
positive     8582
negative     7781
Name: sentiment, dtype: int64

Convert the sentiment to numerical data

In [26]:
def convert_sentiment(sent):
    if(sent=='neutral'):
        return 0
    elif(sent=='positive'):
        return 1
    elif(sent=='negative'):
        return -1

    
df['sentiment']=df.sentiment.apply(convert_sentiment)

<h3>2. Preprocessing the Data</h3>

Lemmatization and Tokenization

In [27]:
def preprocess_text(text):
    # Tokenise words while ignoring punctuation
    tokeniser = RegexpTokenizer(r'\w+')
    tokens = tokeniser.tokenize(text)
    
    # Lowercase and lemmatise 
    lemmatiser = WordNetLemmatizer()
    lemmas = [lemmatiser.lemmatize(token.lower(), pos='v') for token in tokens]
    
    # Remove stop words
    keywords= [lemma for lemma in lemmas if lemma not in stopwords.words('english')]
    return keywords
# Create an instance of TfidfVectorizer


<h4>Vectorizing the Data</h4>

In [28]:
vectoriser = TfidfVectorizer(analyzer=preprocess_text)
# Fit to the data and transform to feature matrix
# df['type']=1
# df_test['type']=2
frame=[df,df_test]
df_new=pd.concat(frame)
X_data_tfidf = vectoriser.fit_transform(df_new['text'].values.astype('U'))
# X_data=pd.DataFrame.sparse.from_spmatrix(X_data_tfidf)



In [29]:
print(X_data_tfidf.shape)
df_test.shape

(31015, 25313)


(3534, 2)

In [30]:
train_dats=X_data_tfidf[0:27481][:]
test_dats=X_data_tfidf[27481:][:]

In [31]:
print(train_dats.shape)
print(test_dats.shape)

(27481, 25313)
(3534, 25313)


In [18]:
# print(X_data.shape)
# print(df.shape)
# abc=df['type']
# print(type(abc))
# X_data['type']=abc.tolist()

(31015, 25313)
(31015, 4)
<class 'pandas.core.series.Series'>


In [32]:
# X_data['sentiment']=df['sentiment'].tolist()

In [33]:
# trainData=X_data[X_data['type']==1]
# testData=X_data[X_data['type']==2]

In [34]:
# import scipy
# trainTextData=trainData.drop(columns=['type','sentiment'])
# testTextData=testData.drop(columns=['type','sentiment'])

In [35]:
# sfd=scipy.sparse.csr_matrix(trainTextData.values)

<h4>Train and Test Split</h4>

In [36]:
X_train, X_test, y_train, y_test = train_test_split(train_dats,df['sentiment'], test_size=0.1, random_state=123)
 

<h3>3. Training the model </h3>

<h4>Declaring the classifier and Cross Validation</h4>

The parameters have been selected using GridSearchCV which is shown below

In [67]:
sgd_clf = SGDClassifier(random_state=123,early_stopping=False,fit_intercept=True,loss='hinge',penalty='l1')
sgf_clf_scores = cross_val_score(sgd_clf, X_train, y_train, cv=5)
print(sgf_clf_scores)
print("Accuracy: %0.2f (+/- %0.2f)" % (sgf_clf_scores.mean(), sgf_clf_scores.std() * 2))


[0.70163736 0.69314736 0.69692681 0.68681763 0.70218358]
Accuracy: 0.70 (+/- 0.01)


<h4>Predicting the Test data</h4>

In [68]:

# vectoriser = TfidfVectorizer(analyzer=preprocess_text)
# X_test_tfidf = vectoriser.fit_transform(X_test.values.astype('U'))
# print(type(X_test_tfidf))
# X_test_tfidf = X_test_tfidf.reindex(labels=X_train_tfidf.columns,axis=1)
sgd_clf=sgd_clf.fit(train_dats,df['sentiment'])
sent_pred=sgd_clf.predict(test_dats)
# print(accuracy_score(y_test,sent_pred))

In [69]:

def convert_inverse(sent):
    if(sent==0):
        return 'neutral'
    elif(sent==1):
        return 'positive'
    elif(sent==-1):
        return 'negative'



    


In [70]:
df_final=pd.DataFrame()
df_final['textID']=df_test['textID']

In [71]:
print(sent_pred)
df_final['sentiment']=sent_pred

[0 1 0 ... 0 1 1]


In [72]:
df_final.head()

,textID,sentiment
0,f87dea47db,0
1,96d74cb729,1
2,eee518ae67,0
3,01082688c6,1
4,33987a8ee5,0


In [73]:
df_final['sentiment']=df_final.sentiment.apply(convert_inverse)

<h4>Saving the output as CSV</h4>

In [74]:
df_final.to_csv(r'C:\Users\udit sharma\Desktop\MSA\sentiment-analysis-msa-phase-2\submission_udit1.csv',columns=['textID','sentiment'])

In [46]:
# # Fit to the data and transform to feature matrix
# X_test_tfidf = vectoriser.transform(df_test['text'].values.astype('U'))
# # print(type(X_test_tfidf))
# X_testing=pd.DataFrame.sparse.from_spmatrix(X_test_tfidf)
# # X_test_tfidf = X_testing.reindex(labels=X_data.columns,axis=1)



In [25]:
# X_testing = X_testing.reindex(labels=X_data.columns,axis=1)
# sent_pred1=sgd_clf.predict(X_test_tfidf)

<h3>4. Parameter Optimization using GridSearchCV</h3>

In [54]:
grid = {'fit_intercept': [True,False],
        'early_stopping': [True, False],
        'loss' : ['hinge', 'log', 'squared_hinge'],
        'penalty' : ['l2', 'l1', 'none']}
search = GridSearchCV(estimator=sgd_clf, param_grid=grid, cv=10)
search.fit(train_dats,df['sentiment'])
search.best_params_

c:\users\udit sharma\appdata\local\programs\python\python38-32\lib\site-packages\sklearn\linear_model\_stochastic_gradient.py:570: ConvergenceWarning: Maximum number of iteration reached before convergence. Consider increasing max_iter to improve the fit.
  warnings.warn("Maximum number of iteration reached before "


{'early_stopping': False,
 'fit_intercept': True,
 'loss': 'hinge',
 'penalty': 'l1'}